## extract specific classes with images

In [1]:
import numpy as np
import cv2
import os
import glob

folder_path = '/data2/bear/n_vptz'
folders = os.listdir(folder_path)

count = 0
count2 = 0
count3 = 0
target_class = '0'
target_class2 = '14'
target_class3 = '24'
replace_class = '5'
limit = 0
error = 0
for folder in folders:
    path = os.path.join(folder_path, folder)
    image_files = glob.glob(f'{path}/*.jpg') + glob.glob(f'{path}/*.png')
    for filename in image_files:
        label = filename[:filename.rfind('.')] + '.txt'
        try:
            f = open(label, 'r')
            line = f.readlines()
            last_count = count
            last_count2 = count2
            last_count3 = count3
            if(len(line) > 1):
                for i in line:
                    if(i[0] != target_class):
                        #break ##?
                        continue ##?
                    
                img_path = path[:path.find('2021_11_03')]+'2024_02_06/person/' + filename[filename.rfind('/')+1:]
                txt_path = img_path[:img_path.find('.')] + '.txt'
                txt = open(txt_path, "w")
                
                img = cv2.imread(filename)
                cv2.imwrite(img_path, img)
                
                for i in line:
                    if(i[0] == target_class):
                        count += 1        
                        txt.write(replace_class+i[i.find(' '):]) ## if you want replace class number
                    elif(i[0] == target_class2):
                        count2 += 1
                        txt.write(target_class2+i[i.find(' '):])
                    elif(i[0] == target_class3):
                        count3 += 1
                        txt.write(target_class3+i[i.find(' '):])

                nofound = 1
                if(last_count==count):
                    nofound = 1
                elif(last_count2==count2):
                    nofound = 1
                elif(last_count3==count3):
                    nofound = 1
                else:
                    nofound = 0
                
                if(nofound):
                    os.remove(txt_path)
                    os.remove(img_path)
                    
                txt.close()
            f.close()
        except:
            error += 1
    #     limit += 1
    #     if(limit == 10000):
    #         break
    # break

print("count: %d\n",count)
print("count2: %d\n",count2)
print("count3: %d\n",count3)
print("error: %d\n",error)

count: %d
 0
count2: %d
 0
count3: %d
 0
error: %d
 0


# target class to set class

In [39]:
import numpy as np
import cv2
import os
import glob

folder = r'/data2/bear/n_Supermarket_CCTV_1'
count = 0
target_class = str('5')  # 변경할 클래스 값

# 새로운 클래스 값
set_class = 4
error = 0

# 폴더 내 이미지 파일 찾기
image_files = glob.glob(f'{folder}/*.jpg') + glob.glob(f'{folder}/*.png') + glob.glob(f'{folder}/*.JPG') + glob.glob(f'{folder}/*.PNG')

txt_list = []
img_list = []

for filename in image_files:
    label = filename[:filename.rfind('.')] + '.txt'
    try:
        f = open(label, 'r')
        lines = f.readlines()
        f.close()  # 파일 읽기 완료 후 바로 닫기

        last_count = count
        modified_lines = []

        # 파일 내 각 라인 확인
        if len(lines) > 0:
            for line in lines:
                # 첫 번째 값이 target_class와 일치하면 수정
                if line[0] == target_class:
                    count += 1
                    # 클래스 값을 set_class로 교체
                    modified_line = str(set_class) + line[line.find(' '):]
                    modified_lines.append(modified_line)
                else:
                    # 클래스가 target_class와 다르면 그대로 유지
                    modified_lines.append(line)
            
            # 이미지 처리
            img_path = filename
            img = cv2.imread(filename)
            cv2.imwrite(img_path, img)

            # 수정된 내용을 덮어쓰지 않고 파일 전체에 다시 작성
            txt_path = img_path[:img_path.find('.')] + '.txt'
            with open(txt_path, "w") as txt_file:
                txt_file.writelines(modified_lines)

            # 객체가 없는 경우 체크
            if last_count == count:
                txt_list.append(txt_path)
                img_list.append(img_path)
    except Exception as e:
        error += 1
        print(f"Error processing {label}: {e}")

# 최종 출력
print(f"count: {count}")
print(f"error: {error}")


count: 5094
error: 0


# all class to set class

In [11]:
import numpy as np
import cv2
import os
import glob

folder = r'/data2/bear/bear_park1'
count = 0
set_class = '0'  # 이 클래스를 제외한 나머지 클래스는 set_class로 변경
error = 0

# 폴더 내 이미지 파일 찾기
image_files = glob.glob(f'{folder}/*.jpg') + glob.glob(f'{folder}/*.png') + glob.glob(f'{folder}/*.JPG') + glob.glob(f'{folder}/*.PNG')

txt_list = []
img_list = []

for filename in image_files:
    label = filename[:filename.rfind('.')] + '.txt'
    try:
        f = open(label, 'r')
        lines = f.readlines()
        f.close()  # 파일 읽기 완료 후 바로 닫기

        last_count = count
        modified_lines = []

        # 파일 내 각 라인 확인
        if len(lines) > 0:
            for line in lines:
                # 클래스 값이 set_class와 같으면 그대로 유지
                if line[0] == set_class:
                    modified_lines.append(line)
                else:
                    # 클래스 값이 set_class와 다르면 set_class로 변경
                    count += 1
                    modified_line = str(set_class) + line[line.find(' '):]
                    modified_lines.append(modified_line)

            # 이미지 처리
            img_path = filename
            img = cv2.imread(filename)
            cv2.imwrite(img_path, img)

            # 수정된 내용을 덮어쓰지 않고 파일 전체에 다시 작성
            txt_path = img_path[:img_path.rfind('.')] + '.txt'
            with open(txt_path, "w") as txt_file:
                txt_file.writelines(modified_lines)

            # 객체가 없는 경우 체크
            if last_count == count:
                txt_list.append(txt_path)
                img_list.append(img_path)
    except Exception as e:
        error += 1
        print(f"Error processing {label}: {e}")

# 최종 출력
print(f"count: {count}")
print(f"error: {error}")


Error processing /data2/bear/bear_park1/PXL_20240614_235026590.txt: [Errno 2] No such file or directory: '/data2/bear/bear_park1/PXL_20240614_235026590.txt'
Error processing /data2/bear/bear_park1/PXL_20240615_001054954.txt: [Errno 2] No such file or directory: '/data2/bear/bear_park1/PXL_20240615_001054954.txt'
Error processing /data2/bear/bear_park1/PXL_20240615_001056870.txt: [Errno 2] No such file or directory: '/data2/bear/bear_park1/PXL_20240615_001056870.txt'
Error processing /data2/bear/bear_park1/PXL_20240615_002835624.txt: [Errno 2] No such file or directory: '/data2/bear/bear_park1/PXL_20240615_002835624.txt'
Error processing /data2/bear/bear_park1/PXL_20240615_002842048.txt: [Errno 2] No such file or directory: '/data2/bear/bear_park1/PXL_20240615_002842048.txt'
Error processing /data2/bear/bear_park1/PXL_20240615_010017239.txt: [Errno 2] No such file or directory: '/data2/bear/bear_park1/PXL_20240615_010017239.txt'
Error processing /data2/bear/bear_park1/PXL_20240615_01112

# count all classes in the folder

In [ ]:
#find all folder
import os
import glob

class_counts = {}  # 클래스 ID를 카운트할 딕셔너리

folder_path = '/data1/datasets/bears/'
folders = os.listdir(folder_path)

for folder in folders:
    path = os.path.join(folder_path, folder)
    image_files = glob.glob(f'{path}/*.jpg') + glob.glob(f'{path}/*.png')
    for filename in image_files:
        label = filename[:filename.rfind('.')] + '.txt'
        txt_file = label
        try:
            with open(txt_file, 'r') as f:
                lines = f.readlines()

            # 각 파일의 내용을 라인별로 처리
            for line in lines:
                class_id = line.split(' ')[0]  # 클래스 ID는 첫 번째 항목
                
                # 클래스 ID 카운트
                if class_id in class_counts:
                    class_counts[class_id] += 1
                else:
                    class_counts[class_id] = 1
        except Exception as e:
            print(f"Error reading {txt_file}: {e}")

# 최종 클래스 ID 카운트 출력
for class_id, count in class_counts.items():
    print(f"Class {class_id}: {count} instances")


Processing folder: /data2/bear/bear_aihub
Processing folder: /data2/bear/bear_gray
Processing folder: /data2/bear/bear_park1
Processing folder: /data2/bear/bear_park2
Processing folder: /data2/bear/bear_park3
Processing folder: /data2/bear/bear_white
Processing folder: /data2/bear/bear_you1
Class 0: 103914 instances


In [ ]:
# find some folder
import os
import glob

# 각 폴더들의 이름을 리스트로 입력받습니다.
folders = [r'/data1/datasets/bears']  # 예시로 폴더 경로 리스트

for folder in folders:
    print(f"Processing folder: {folder}")    
    # 폴더 내 이미지 파일들의 텍스트 파일(.txt) 경로를 찾습니다.
    txt_files = glob.glob(f'{folder}/*.txt')
    for txt_file in txt_files:
        try:
            txt_file
            with open(txt_file, 'r') as f:
                lines = f.readlines()

            # 각 파일의 내용을 라인별로 처리
            for line in lines:
                class_id = line.split(' ')[0]  # 클래스 ID는 첫 번째 항목
                
                # 클래스 ID 카운트
                if class_id in class_counts:
                    class_counts[class_id] += 1
                else:
                    class_counts[class_id] = 1
        except Exception as e:
            continue
            print(f"Error reading {txt_file}: {e}")

# 최종 클래스 ID 카운트 출력
for class_id, count in class_counts.items():
    print(f"Class {class_id}: {count} instances")


Processing folder: /data2/bear/bear_aihub
Processing folder: /data2/bear/bear_gray
Processing folder: /data2/bear/bear_park1
Processing folder: /data2/bear/bear_park2
Processing folder: /data2/bear/bear_park3
Processing folder: /data2/bear/bear_white
Processing folder: /data2/bear/bear_you1
Files containing classes other than 0:
- /data2/bear/bear_park1/PXL_20240614_234740050.MP.txt
- /data2/bear/bear_park1/PXL_20240614_234745259.MP.txt
- /data2/bear/bear_park1/PXL_20240614_234745968.MP.txt
- /data2/bear/bear_park1/PXL_20240614_234746728.MP.txt
- /data2/bear/bear_park1/PXL_20240615_002812679.MP.txt
- /data2/bear/bear_park1/PXL_20240615_002813503.MP.txt
- /data2/bear/bear_park1/PXL_20240615_002815180.MP.txt
- /data2/bear/bear_park1/PXL_20240615_002829136.MP.txt
- /data2/bear/bear_park1/PXL_20240615_002830424.MP.txt
- /data2/bear/bear_park1/PXL_20240615_002832583.MP.txt
- /data2/bear/bear_park1/PXL_20240614_234730276.MP.txt
- /data2/bear/bear_park1/PXL_20240615_002858019.MP.txt
- /data2/

# delete target class

In [25]:
import os
import glob

# 각 폴더들의 이름을 리스트로 입력받습니다.
folders = [r'/data2/bear/n_vocperson', r'/data2/bear/n_vptz', r'/data2/bear/n_youtube_dataset_han', r'/data2/bear/n_youtube_fun1', r'/data2/bear/n_Supermarket_CCTV_1', r'/data2/bear/n_Supermarket_CCTV_2', r'/data2/bear/n_Supermarket_CCTV_3', r'/data2/bear/n_Supermarket_CCTV_4', r'/data2/bear/n_Supermarket_CCTV_5', r'/data2/bear/n_Supermarket_CCTV_6', r'/data2/bear/n_vertical', r'/data2/bear/n_VisDrone', r'/data2/bear/n_VOC']
target_class = '4'  # 제거하려는 클래스 ID

for folder in folders:
    print(f"Processing folder: {folder}")
    
    # 폴더 내 텍스트 파일(.txt) 경로를 찾습니다.
    txt_files = glob.glob(f'{folder}/*.txt')
    
    for txt_file in txt_files:
        try:
            with open(txt_file, 'r') as f:
                lines = f.readlines()

            # 특정 클래스 ID 라인 제거
            modified_lines = [line for line in lines if line.split(' ')[0] != target_class]
            
            # 만약 라인이 수정되었다면 파일에 다시 작성
            if len(modified_lines) != len(lines):
                with open(txt_file, 'w') as f:
                    f.writelines(modified_lines)
                print(f"Class {target_class} labels removed in file: {txt_file}")
        except Exception as e:
            print(f"Error processing {txt_file}: {e}")

print("Processing complete.")


Processing folder: /data2/bear/n_vocperson
Class 4 labels removed in file: /data2/bear/n_vocperson/000218.txt
Class 4 labels removed in file: /data2/bear/n_vocperson/001542.txt
Class 4 labels removed in file: /data2/bear/n_vocperson/002180.txt
Class 4 labels removed in file: /data2/bear/n_vocperson/007393.txt
Class 4 labels removed in file: /data2/bear/n_vocperson/002696.txt
Class 4 labels removed in file: /data2/bear/n_vocperson/2010_005130.txt
Class 4 labels removed in file: /data2/bear/n_vocperson/008045.txt
Class 4 labels removed in file: /data2/bear/n_vocperson/2009_001207.txt
Class 4 labels removed in file: /data2/bear/n_vocperson/2009_004403.txt
Class 4 labels removed in file: /data2/bear/n_vocperson/003253.txt
Class 4 labels removed in file: /data2/bear/n_vocperson/2008_007528.txt
Class 4 labels removed in file: /data2/bear/n_vocperson/2011_007079.txt
Class 4 labels removed in file: /data2/bear/n_vocperson/003859.txt
Class 4 labels removed in file: /data2/bear/n_vocperson/00781